In [4]:
import pandas as pd

df = pd.read_csv('C:/Users/ekart/casual_inference/communications.csv', sep=';')

In [5]:
df.head()

,communication_id,communication_dt,employee_id,communication_score,util_flg
0,177074281189583840,2023-02-07 00:00:00,269d837a-fada-308d-d4ae-ab28ca2d57e4,85,0
1,189925315406757324,2023-07-29 00:00:00,e6384711-4917-13d2-9bc6-3fc5eeb5ba4f,100,0
2,321818238739717242,2023-09-06 00:00:00,ab541d87-4c7b-c19a-b776-42849e02b89f,43,1
3,192259329962028989,2023-06-19 00:00:00,0a0a0c8a-aa00-ade5-0f74-a3f0ca981ed7,100,1
4,705274671052151857,2023-08-11 00:00:00,f93882cb-d8fc-7fb7-94c1-011d63be6fb6,46,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5345246 entries, 0 to 5345245
Data columns (total 5 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   communication_id     int64 
 1   communication_dt     object
 2   employee_id          object
 3   communication_score  int64 
 4   util_flg             int64 
dtypes: int64(3), object(2)
memory usage: 203.9+ MB


In [7]:
df.describe()

,communication_id,communication_score,util_flg
count,5.345246e+06,5.345246e+06,5.345246e+06
mean,5.001990e+17,5.855952e+01,7.524716e-01
std,2.917627e+17,3.707632e+01,4.315763e-01
min,1.052382e+09,0.000000e+00,0.000000e+00
25%,2.437200e+17,2.900000e+01,1.000000e+00
50%,5.004468e+17,6.700000e+01,1.000000e+00
75%,7.565783e+17,1.000000e+02,1.000000e+00
max,9.999998e+17,1.000000e+02,1.000000e+00


In [1]:
from dowhy import CausalModel

# Загружаем данные (у вас уже должны быть загружены данные)
employees_df = pd.read_csv('C:/Users/ekart/casual_inference/employees.csv', sep=';')
courses_passing_df = pd.read_csv('C:/Users/ekart/casual_inference/courses_passing.csv', sep=';')

# Объединение данных по идентификатору сотрудника
merged_df = pd.merge(courses_passing_df, employees_df, on='employee_id')

# Пример причинно-следственного анализа: влияние работы онлайн на успеваемость по курсам
model = CausalModel(
    data=merged_df,
    treatment='work_online_flg',
    outcome='pass_frac',
    common_causes=['age', 'region', 'exp_days', 'sex']
)

# Идентификация эффекта
identified_estimand = model.identify_effect()

# Оценка эффекта
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.linear_regression")

# Вывод результатов
print(estimate)

*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
        d                                               
──────────────────(E[pass_frac|sex,region,age,exp_days])
d[work_online_flg]                                      
Estimand assumption 1, Unconfoundedness: If U→{work_online_flg} and U→pass_frac then P(pass_frac|work_online_flg,sex,region,age,exp_days,U) = P(pass_frac|work_online_flg,sex,region,age,exp_days)

## Realized estimand
b: pass_frac~work_online_flg+sex+region+age+exp_days
Target units: ate

## Estimate
Mean value: -0.0017541446321209797



* Identified estimand (Идентифицированная оценка): Это математическое выражение оценки причинного эффекта, которое DoWhy сгенерировало на основе вашей модели данных и указанных вами причинных связей.

* Estimand expression (Выражение оценки): Показывает, что оценка эффекта будет рассчитываться как производная ожидаемого значения pass_frac по work_online_flg, контролируя переменные sex, region, age, exp_days.

* Estimand assumption (Предположение оценки): Предположение об отсутствии путаницы, означающее, что нет скрытых переменных U, которые одновременно влияют на work_online_flg и pass_frac.

* Realized estimand (Реализованная оценка): Указывает, какие переменные включены в линейную регрессионную модель для оценки воздействия.

* Mean value (Среднее значение): Это численное значение оценки эффекта. В вашем случае -0.0017541446321209797 означает, что переход от не работающих онлайн к работающим онлайн связан со снижением доли прохождения курса на величину 0.0017 или 0.17%. Это очень маленькое изменение, что может означать, что эффект работы онлайн на успеваемость по курсам минимален.


Таким образом, если предположение о непутанице верно, и модель адекватно специфицирована, то работа онлайн не оказывает значительного прямого влияния на долю прохождения курса среди сотрудников. 

In [2]:
print(dowhy.__version__)
print(sklearn.__version__)


0.11.1
1.1.3
